In [1]:
import pandas as pd
import numpy as np
np.random.seed(1)

In [2]:
df = pd.read_csv("../input_data/data.csv")

In [3]:
X = df.drop('xAttack',axis=1)
y = df['xAttack']

In [4]:
X = np.asarray(X)
X = (X-np.mean(X,axis=0))/(np.std(X,axis=0))

In [5]:
X.shape

(125973, 29)

In [6]:
def traintestvalidatesplit(data):
    x,y = data[0], data[1]
    n = x.shape[0]
    k = int(n*0.8)
    x_train = x[:k]
    y_train = y[:k]
    x_val = x[k:]
    y_val = y[k:]
    return [x_train,y_train],[x_val,y_val]

In [7]:
def linear(x):
    return x

In [8]:
def linear_derivative(x):
    return np.where(x > 0, 1.0, 1.0)

In [9]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [10]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [11]:
def relu(x):
    return np.where(x > 0, 1.0, 0.0)

In [12]:
def relu_derivative(x):
    return np.where(x > 0, 1.0, 0.0)

In [13]:
def tanh(x):
    return (2/(1+np.exp(-2*x))) - 1

In [14]:
def tanh_derivative(x):
    return 1 - (x * x) 

In [15]:
class NN:
    def __init__(self,layerSizes,activation,learningrate = 0.01):
        self.shape = layerSizes
        self.activation = activation
        self.learningrate = learningrate
        self.activations = {'sigmoid':[sigmoid,sigmoid_derivative],
                           'relu':[relu,relu_derivative],
                           'tanh':[tanh,tanh_derivative],
                           'linear':[linear,linear_derivative]}
        n = len(layerSizes)
        self.layers = []
        self.layers.append(np.ones(self.shape[0]))
        for i in range(1,n):
            self.layers.append(np.ones(self.shape[i]))
        self.weights = []
        for i in range(n-1):
            temp = np.zeros((self.layers[i].size,self.layers[i+1].size), dtype = 'd')
            self.weights.append(np.random.randn(*temp.shape))
        self.derivative = [0,]*len(self.weights)

    def forwardpass(self,data):
        self.layers[0] = data
        for i in range(1,len(self.shape)):
            self.layers[i][...] = self.activations[self.activation][0](np.dot(self.layers[i-1],self.weights[i-1]))
        return self.layers[-1]


    def backpropogation(self, target, momentum=0.1):
        error = target - self.layers[-1]
        weight_deltas = []
        weight_delta = error*self.activations[self.activation][1](self.layers[-1])
        weight_deltas.append(weight_delta)

        for i in range(len(self.shape)-2,0,-1):
            weight_delta = np.dot(weight_deltas[0],self.weights[i].T)*self.activations[self.activation][1](self.layers[i])
            weight_deltas.insert(0,weight_delta)
            
        for i in range(len(self.weights)):
            layer = np.atleast_2d(self.layers[i])
            weight_delta = np.atleast_2d(weight_deltas[i])
            der = np.dot(layer.T,weight_delta)
            self.weights[i] += self.learningrate*der + momentum*self.derivative[i]
            self.derivative[i] = der

        return (error**2).mean()

In [16]:
def trainMLP(network,samples, epochs=10, momentum=0.1):
    error_set = []
    for i in range(epochs):
        print('Epoch: ', i+1)
        n = samples[0].shape[0]
        error = 0
        for j in range(n):
            out = network.forwardpass(samples[0][j] )
            error += network.backpropogation( samples[1][j], momentum )
        error_set.append(error/n)
        print('Training error',error/n)
    return error_set, error/n

In [19]:
nn = NN([X.shape[1],14,X.shape[1]],'relu',0.01)
epochs = 100
error_set, finalerror = trainMLP(nn,[X,X],epochs)

Epoch:  1
Training error 0.8737579085698337
Epoch:  2
Training error 0.873874092819078
Epoch:  3
Training error 0.874109560077602
Epoch:  4
Training error 0.8704756339086077
Epoch:  5
Training error 0.8756422834233158
Epoch:  6
Training error 0.8754237190389235
Epoch:  7
Training error 0.8811132232154536
Epoch:  8
Training error 0.883036349734231
Epoch:  9
Training error 0.8835228678992805
Epoch:  10
Training error 0.8837930587504453
Epoch:  11
Training error 0.8839652947882175
Epoch:  12
Training error 0.8840054670418389
Epoch:  13
Training error 0.8842223989408962
Epoch:  14
Training error 0.8843915760829952
Epoch:  15
Training error 0.8844020505489533
Epoch:  16
Training error 0.8844173543318687
Epoch:  17
Training error 0.88415935942823
Epoch:  18
Training error 0.8837051567203176
Epoch:  19
Training error 0.8837055865138633
Epoch:  20
Training error 0.8837183353204636
Epoch:  21
Training error 0.883713842032243
Epoch:  22
Training error 0.8837060238717211
Epoch:  23
Training error

In [20]:
reduced_data = pd.DataFrame(np.dot(X,nn.weights[0]))

In [21]:
reduced_data.insert(loc=13, column='class', value=y)
reduced_data.shape

(125973, 15)

In [22]:
pd.DataFrame(reduced_data).to_csv("../input_data/reducedData_a.csv",index=False)